<a href="https://colab.research.google.com/github/Andre6o6/mlcourse-2019/blob/master/Task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Factorization machine from this paper: 
https://www.csie.ntu.edu.tw/~b97053/paper/Rendle2010FM.pdf

In [0]:
!wget https://archive.org/download/nf_prize_dataset.tar/nf_prize_dataset.tar.gz

--2019-10-22 11:23:00--  https://archive.org/download/nf_prize_dataset.tar/nf_prize_dataset.tar.gz
Resolving archive.org (archive.org)... 207.241.224.2
Connecting to archive.org (archive.org)|207.241.224.2|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://ia800205.us.archive.org/7/items/nf_prize_dataset.tar/nf_prize_dataset.tar.gz [following]
--2019-10-22 11:23:01--  https://ia800205.us.archive.org/7/items/nf_prize_dataset.tar/nf_prize_dataset.tar.gz
Resolving ia800205.us.archive.org (ia800205.us.archive.org)... 207.241.230.25
Connecting to ia800205.us.archive.org (ia800205.us.archive.org)|207.241.230.25|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 697552028 (665M) [application/octet-stream]
Saving to: ‘nf_prize_dataset.tar.gz’

nf_prize_dataset.ta 100%[===================>] 665.24M  2.06MB/s    in 11m 8s  

2019-10-22 11:34:10 (1019 KB/s) - ‘nf_prize_dataset.tar.gz’ saved [697552028/697552028]



In [1]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
#!cp nf_prize_dataset.tar.gz '/gdrive/My Drive/data/'
!cp '/gdrive/My Drive/data/nf_prize_dataset.tar.gz' .

In [0]:
!tar -xzf nf_prize_dataset.tar.gz
!tar -xf download/training_set.tar

In [0]:
import os
import numpy as np

transactions = np.zeros((100480507, 3), dtype=int)
i = 0

root = 'training_set/'
for filename in os.listdir(root):
    file = root + filename
    with open(file) as f:
        lines = f.readlines()
        movie_idx = int(lines[0].split(':')[0])
        for line in lines[1:]:
            user_idx, score, _ = line.split(',')
            user_idx, score = int(user_idx), int(score)
            transactions[i] = movie_idx, user_idx, score
            i+=1

# Used for onehot
user_idxs = {x:i for i,x in enumerate(set(transactions[:, 1]))}

In [0]:
np.save('transactions.npy', transactions)

In [0]:
import os
import numpy as np
transactions = np.load('transactions.npy')

In [0]:
p = np.random.permutation(len(transactions))
transactions = transactions[p]

In [40]:
transactions[:10]

array([[  17513, 1505912,       4],
       [   9125, 2011196,       5],
       [  13949,  576329,       5],
       [  17482,  337129,       2],
       [   2585, 2622076,       4],
       [   3017, 1981427,       5],
       [  16203, 1419139,       3],
       [   6408, 1398903,       3],
       [  10123,  429876,       3],
       [   3782, 1423440,       3]])

In [0]:
#PREPROCESS
user_idxs = {x:i for i,x in enumerate(set(transactions[:, 1]))}

X = np.zeros((transactions.shape[0], 2))
X[:, 0] = transactions[:, 0] - 1    #Movie id
X[:, 1] = 17770 + np.vectorize(user_idxs.get)(transactions[:, 1])   #User id, offset by n_movies=17770
X = X.astype(int)
y = (transactions[:, 2] - 3) / 2    #Rating, centered and scaled to [-1, 1]

In [66]:
import gc
gc.collect()

1334

In [0]:
def rmse_score(y_true, y_pred):
    return np.mean(np.square(y_true - y_pred))**0.5

def r2_score(y, y_pred):
    y_avg = y.mean()
    ss_total = np.sum(np.square(y - y_avg))
    ss_err = np.sum(np.square(y - y_pred))
    return 1 - ss_err/ss_total

In [0]:
class KFolds:
    def __init__(self, X, y, k=5):
        self.k = k
        self.X_folds = [X[i::k] for i in range(k)]
        self.y_folds = [y[i::k] for i in range(k)]
        
    def get_fold(self, fold_i):
        
        X_cv = self.X_folds[fold_i]
        y_cv = self.y_folds[fold_i]
        
        X_train = np.concatenate([self.X_folds[i] for i in range(self.k) if i != fold_i])
        y_train = np.concatenate([self.y_folds[i] for i in range(self.k) if i != fold_i])
        
        return X_train, y_train, X_cv, y_cv


In [0]:
import math

class FactorizationMachineSparse:
    def __init__(self, n, k):
        self.w0 = 0
        self.w = 0.01*np.random.randn(n)
        self.v = 0.01*np.random.randn(n,k)

        self.lr = 0.01
        # Adam hyperparams
        self.beta1 = 0.9
        self.beta2 = 0.999
        # Adam moments
        self.v_dw0 = np.zeros_like(self.w0)
        self.s_dw0 = np.zeros_like(self.w0)
        self.v_dw = np.zeros_like(self.w)
        self.s_dw = np.zeros_like(self.w)
        self.v_dv = np.zeros_like(self.v)
        self.s_dv = np.zeros_like(self.v)
        # 
        self.t = 0
        self.eps = 1e-8

        #cache stuff to use in backward pass
        self.x_batch = None
        self.v_dot_x = None

    def forward(self, x):
        '''
        x - tensor [b x 2], where x_i = (movie_id, user_id)
        '''
        
        self.x_batch = x
        self.v_dot_x = np.sum(self.v[x], axis=1)
        
        return self.w0   \
               + np.sum(self.w[x], axis=1)   \
               + 0.5 * np.sum(np.square(self.v_dot_x) - np.sum(np.square(self.v[x]), axis=1), axis=1)
         
    def backward(self, dLdy):
        # TODO vstack x_batch[:,0 and 1] and dldy x2 (if torch can't iterate w/ tuples)
        if self.x_batch is None:
            assert 0, 'Call forward first'

        dLdw0 = np.mean(dLdy)

        dLdw = np.zeros(n)
        for x, dLdyi in zip(self.x_batch, dLdy):
            dLdw[x] +=  dLdyi
        dLdw /= dLdy.shape[0]

        dLdv = np.zeros((n,k))
        for x, v_dot_xi, dLdyi in zip(self.x_batch, self.v_dot_x, dLdy):
            dLdv[x] += dLdyi * (v_dot_xi - self.v[x])
        dLdv /= dLdy.shape[0]

        #ADAM
        self.v_dw0 = self.beta1 * self.v_dw0 + (1 - self.beta1) * dLdw0
        self.s_dw0 = self.beta2 * self.s_dw0 + (1 - self.beta2) * dLdw0 * dLdw0
        self.v_dw = self.beta1 * self.v_dw + (1 - self.beta1) * dLdw
        self.s_dw = self.beta2 * self.s_dw + (1 - self.beta2) * dLdw * dLdw
        self.v_dv = self.beta1 * self.v_dv + (1 - self.beta1) * dLdv
        self.s_dv = self.beta2 * self.s_dv + (1 - self.beta2) * dLdv * dLdv
        #ADAM: correct moments
        self.t+=1
        bias_correction1 = 1 - self.beta1**self.t
        bias_correction2 = 1 - self.beta2**self.t

        step_size = self.lr / bias_correction1

        denom_dw0 = np.sqrt(self.s_dw0) / math.sqrt(bias_correction2) + self.eps
        denom_dw = np.sqrt(self.s_dw) / math.sqrt(bias_correction2) + self.eps
        denom_dv = np.sqrt(self.s_dv) / math.sqrt(bias_correction2) + self.eps

        self.w0 -= step_size * self.v_dw0/denom_dw0
        self.w -= step_size * self.v_dw/denom_dw
        self.v -= step_size * self.v_dv/denom_dv

        '''self.w0 -= self.lr * dLdw0
        self.w -= self.lr * dLdw
        self.v -= self.lr * dLdv'''
        
        #Clear cache
        self.x_batch = None
        self.v_dot_x = None

In [0]:
class MSE:
    def __init__(self):
        self.err = None
    def forward(self, y_true, y_pred):
        self.err = y_true - y_pred
        return np.mean(np.square(self.err))
    def backward(self):
        if self.err is None:
            assert 0, 'Call forward first'
        return -2 * self.err	

In [0]:
def get_batch(X,y, i):
  return X[i*batch_size:(i+1)*batch_size], \
         y[i*batch_size:(i+1)*batch_size]

In [0]:
n_movies = 17770
n_users = len(user_idxs)

n = n_movies + n_users
k = 3

criterion = MSE()
kfold = KFolds(X, y)

In [0]:
from tqdm import tqdm

rmses = []
r2s = []

epochs = 1
for fold_i in range(kfold.k):
    model = FactorizationMachineSparse(n,k)

    print(" Train on fold {}".format(fold_i+1))
    X_train, y_train, X_test, y_test = kfold.get_fold(fold_i)

    batch_size = 100000
    iters = X_train.shape[0] // batch_size
    if (X_train.shape[0] % batch_size > 0):
        iters += 1

    for epoch in range(epochs):
        print("Epoch", epoch+1)
        running_loss = 0
        running_r2 = 0
        for i in range(iters):
            X_batch, y_batch = get_batch(X_train,y_train, i)

            y_pred = model.forward(X_batch)
            loss = criterion.forward(y_batch, y_pred)
            dLdy = criterion.backward()
            model.backward(dLdy)

            running_loss += loss
            r2 = r2_score(y_batch, y_pred)
            running_r2 += r2

            if (i+1) % 10 == 0:
                print("[{}]({}) l = {} | r2 = {}".format(epoch, i*batch_size, loss, r2))
        
        running_loss /= X_train.shape[0]/batch_size
        running_r2 /= X_train.shape[0]/batch_size
        print()
        print("MSE = {}, R2 = {}".format(running_loss, running_r2))

    #TEST
    print(" Test on {} fold".format(fold_i+1))

    y_pred = np.zeros(y_test.shape)
    for i in range(iters):
        X_batch, _ = get_batch(X_test,y_test, i)
        y_pred[i*batch_size:(i+1)*batch_size] = model.forward(X_batch)

    print()
    rmses.append(rmse_score(y_test, y_pred))
    r2s.append(r2_score(y_test, y_pred))
    print("RMSE = {}, R2 = {}".format(rmses[-1], r2s[-1]))